In [1]:
from functools import partial
import os
import tempfile
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from ray import tune
from ray import train
from ray.train import Checkpoint, get_checkpoint
from ray.tune.schedulers import ASHAScheduler
import ray.cloudpickle as pickle
from ultralytics import YOLO

In [2]:
#best_model = YOLO("best_model.pt")

#print(f"The task type of the model is: {best_model.task}")

In [3]:
TASK2METRIC = {
    "detect": "metrics/mAP50(B)", #detect 지표를 mAP50-95에서 50으로 변경하기 위함
    "segment": "metrics/mAP50-95(M)",
    "classify": "metrics/accuracy_top1",
    "pose": "metrics/mAP50-95(P)",
    "obb": "metrics/mAP50-95(B)",
}

In [4]:
experiment_name = 'tune'
work_name = 'raytune' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{work_name}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

# Load a YOLOv8n model
best_model_tuning = YOLO("yolov8n.pt")

task = best_model_tuning.task
metric = TASK2METRIC[task] 

result_grid = best_model_tuning.tune(
    data="/home/under1/Detect/jeongui/sagittal/sagittal_t1wce_2_class.yaml",
    epochs=200,
    use_ray=True,
    name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'),
    gpu_per_trial=1  # 각 트라이얼에 1개의 GPU 할당
)

(_tune pid=3940) New https://pypi.org/project/ultralytics/8.2.51 available 😃 Update with 'pip install -U ultralytics'
(_tune pid=3937) Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
(_tune pid=3938) engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/under1/Detect/jeongui/sagittal/sagittal_t1wce_2_class.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=tune_raytune, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=F

  0%|          | 0.00/6.25M [00:00<?, ?B/s]
 11%|█▏        | 728k/6.25M [00:00<00:00, 7.37MB/s]
 14%|█▍        | 920k/6.25M [00:00<00:02, 2.25MB/s]
100%|██████████| 6.25M/6.25M [00:01<00:00, 4.91MB/s]
 96%|█████████▌| 6.00M/6.25M [00:01<00:00, 4.76MB/s]
100%|██████████| 6.25M/6.25M [00:01<00:00, 3.60MB/s]
100%|██████████| 6.25M/6.25M [00:02<00:00, 3.07MB/s]
100%|██████████| 6.25M/6.25M [00:02<00:00, 2.99MB/s]
train: Scanning /home/under1/Detect/dataset/sagittal/labels/train...:   0%|          | 0/212 [00:00<?, ?it/s]
  0%|          | 0.00/6.25M [00:00<?, ?B/s] [repeated 3x across cluster]
 88%|████████▊ | 5.50M/6.25M [00:02<00:00, 4.23MB/s] [repeated 60x across cluster]
 81%|████████  | 5.05M/6.25M [00:02<00:00, 4.06MB/s]
train: Scanning /home/under1/Detect/dataset/sagittal/labels/train... 61 images, 0 backgrounds, 0 corrupt:  29%|██▉       | 61/212 [00:00<00:00, 593.81it/s]


(_tune pid=3940) AMP: checks passed ✅
(_tune pid=3939) Overriding model.yaml nc=80 with nc=2 [repeated 3x across cluster]
(_tune pid=3939)                    from  n    params  module                                       arguments                      [repeated 3x across cluster]
(_tune pid=3939)  19                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]               [repeated 21x across cluster]
(_tune pid=3939)   8                  -1  1    460288  ultralytics.nn.modules.block.C2f             [256, 256, 1, True]            [repeated 12x across cluster]
(_tune pid=3939)   9                  -1  1    164608  ultralytics.nn.modules.block.SPPF            [256, 256, 5]                  [repeated 3x across cluster]
(_tune pid=3939)  13                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']           [repeated 6x across cluster]
(_tune pid=3939)  20             [-1, 9]  1         0  ultralytics.nn.module

train: Scanning /home/under1/Detect/dataset/sagittal/labels/train... 135 images, 0 backgrounds, 0 corrupt:  64%|██████▎   | 135/212 [00:00<00:00, 665.39it/s]
train: Scanning /home/under1/Detect/dataset/sagittal/labels/train... 212 images, 0 backgrounds, 0 corrupt: 100%|██████████| 212/212 [00:00<00:00, 693.96it/s]
train: Scanning /home/under1/Detect/dataset/sagittal/labels/train.cache... 212 images, 0 backgrounds, 0 corrupt: 100%|██████████| 212/212 [00:00<?, ?it/s]
(_tune pid=3940) INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.11 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


(_tune pid=3940) albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))
(_tune pid=3939) albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))
(_tune pid=3938) albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))
(_tune pid=3937) albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))
(_tune pid=3939) Plotting labels to runs/detect/tune/tune_raytune/labels.jpg... 
(_tune pid=3937) AMP: checks passed ✅ [repeated 3x across cluster]
(_tune pid=3938) optimizer: 'optimizer=auto' found, ignoring 'lr0=0.06716053698174035' and 'momentum=0.7887290511194522' and determining

...

  0%|          | 0/14 [00:00<?, ?it/s]
    200/200       2.1G    0.01066     0.8491     0.9929         16        640:   0%|          | 0/14 [00:00<?, ?it/s]
    199/200      2.13G    0.01044     0.8645      1.004          4        640: 100%|██████████| 14/14 [00:01<00:00, 13.08it/s] [repeated 9x across cluster]
    199/200       2.1G    0.01074     0.8971      1.006         17        640:  57%|█████▋    | 8/14 [00:00<00:00, 12.68it/s] [repeated 8x across cluster]
    199/200       2.1G    0.01042     0.8755     0.9917         18        640:  71%|███████▏  | 10/14 [00:00<00:00, 13.09it/s] [repeated 5x across cluster]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/2 [00:00<?, ?it/s] [repeated 3x across cluster]
    200/200       2.1G    0.01104     0.9252      1.027         17        640:  14%|█▍        | 2/14 [00:00<00:01, 11.76it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  50

(_tune pid=9686) 
(_tune pid=9686) 200 epochs completed in 0.118 hours.
(_tune pid=9686) Optimizer stripped from runs/detect/tune/tune_raytune/weights/last.pt, 6.3MB
(_tune pid=9686) Optimizer stripped from runs/detect/tune/tune_raytune/weights/best.pt, 6.3MB
(_tune pid=9686) 
(_tune pid=9686) Validating runs/detect/tune/tune_raytune/weights/best.pt...
(_tune pid=9686) Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
(_tune pid=9686) Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  50%|█████     | 1/2 [00:00<00:00,  8.62it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  8.35it/s]


(_tune pid=9686)               negative         30         30      0.725        0.7      0.832       0.57
(_tune pid=9686)               positive         22         24      0.574      0.786      0.706       0.46
(_tune pid=9686) Speed: 0.1ms preprocess, 0.6ms inference, 0.0ms loss, 0.6ms postprocess per image
(_tune pid=9686) Results saved to runs/detect/tune/tune_raytune
(_tune pid=9686)                    all         52         54       0.65      0.743      0.769      0.515 [repeated 2x across cluster]
(_tune pid=9686)       Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


2024-07-08 14:25:50,132	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/under1/Detect/jeongui/sagittal/runs/detect/tune/_tune_2024-07-08_14-14-59' in 0.0176s.
2024-07-08 14:25:50,150	INFO tune.py:1041 -- Total run time: 650.72 seconds (650.60 seconds for the tuning loop).


In [5]:
for i, result in enumerate(result_grid):
    print(f"Trial #{i}: Configuration: {result.config}, Last Reported Metrics: {result.metrics}")

Trial #0: Configuration: {'lr0': 0.05304189757057075, 'lrf': 0.28304690417335604, 'momentum': 0.8179777597116429, 'weight_decay': 0.00023996813277645668, 'warmup_epochs': 1.2561066368720768, 'warmup_momentum': 0.892720552057496, 'box': 0.0437768295999493, 'cls': 3.4086117193344525, 'hsv_h': 0.012592897671193938, 'hsv_s': 0.7792091715919305, 'hsv_v': 0.18103203543178772, 'degrees': 20.107615888598982, 'translate': 0.8706738667427756, 'scale': 0.36652640553066396, 'shear': 5.09382762363939, 'perspective': 0.0001437054405297903, 'flipud': 0.24086343011590539, 'fliplr': 0.3779710153755035, 'bgr': 0.03809541720109555, 'mosaic': 0.0681544837522512, 'mixup': 0.18665945523555827, 'copy_paste': 0.33829706974170426, 'data': '/home/under1/Detect/jeongui/sagittal/sagittal_t1wce_2_class.yaml', 'epochs': 200, 'name': 'tune/tune_raytune'}, Last Reported Metrics: {'metrics/precision(B)': 0.35165, 'metrics/recall(B)': 0.38582, 'metrics/mAP50(B)': 0.29641, 'metrics/mAP50-95(B)': 0.1122, 'val/box_loss': 

In [6]:
# Extract the best trial configuration
best_result = result_grid.get_best_result(metric=metric, mode="max")
best_config = best_result.config
print("Best hyperparameters found were: ", best_config)

Best hyperparameters found were:  {'lr0': 0.09027564811498429, 'lrf': 0.8262179230956304, 'momentum': 0.8996063252230653, 'weight_decay': 0.00040460786317717966, 'warmup_epochs': 3.77529417786332, 'warmup_momentum': 0.3226077439946397, 'box': 0.097641148015046, 'cls': 0.9950129642929877, 'hsv_h': 0.02676374357201755, 'hsv_s': 0.33127236878657734, 'hsv_v': 0.21401850716143195, 'degrees': 6.313751093841468, 'translate': 0.13631864411629308, 'scale': 0.17474283455316866, 'shear': 5.838978608184045, 'perspective': 0.0004600444691104501, 'flipud': 0.04916985137226626, 'fliplr': 0.056422501426107385, 'bgr': 0.3529938721648831, 'mosaic': 0.05769751977237525, 'mixup': 0.10596540766930995, 'copy_paste': 0.4315639913404711, 'data': '/home/under1/Detect/jeongui/sagittal/sagittal_t1wce_2_class.yaml', 'epochs': 200, 'name': 'tune/tune_raytune'}


In [7]:
experiment_name = 'train'
work_name = 'raytune' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{work_name}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

# 'data' 항목 제거
if 'data' in best_config:
    del best_config['data']
if 'name' in best_config:
    del best_config['name']

# 최고의 하이퍼파라미터로 모델 재훈련
best_model = YOLO("yolov8n.pt")  # 모델 재초기화
best_results = best_model.train(data="sagittal_t1wce_2_class.yaml",
                                optimizer='SGD',
                                augment=True,
                                name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'),
                                **best_config)

New https://pypi.org/project/ultralytics/8.2.51 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=sagittal_t1wce_2_class.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train_raytune2, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, s

train: Scanning /home/under1/Detect/dataset/sagittal/labels/train.cache... 212 images, 0 backgrounds, 0 corrupt: 100%|██████████| 212/212 [00:00<?, ?it/s]
INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.11 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /home/under1/Detect/dataset/sagittal/labels/val.cache... 52 images, 0 backgrounds, 0 corrupt: 100%|██████████| 52/52 [00:00<?, ?it/s]


Plotting labels to runs/detect/train/train_raytune2/labels.jpg... 
optimizer: SGD(lr=0.09027564811498429, momentum=0.8996063252230653) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.00040460786317717966), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train/train_raytune2
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    200/200      2.12G    0.01358      1.498      1.262          4        640: 100%|██████████| 14/14 [00:01<00:00, 11.19it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  7.97it/s]

                   all         52         54      0.452      0.746      0.514      0.295



200 epochs completed in 0.107 hours.
Optimizer stripped from runs/detect/train/train_raytune2/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/train_raytune2/weights/best.pt, 6.3MB

Validating runs/detect/train/train_raytune2/weights/best.pt...
Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.39it/s]


                   all         52         54      0.609       0.63      0.619      0.385
              negative         30         30      0.781      0.594      0.755      0.488
              positive         22         24      0.437      0.667      0.484      0.281
Speed: 0.1ms preprocess, 13.1ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/train/train_raytune2


In [8]:
best_results.results_dict

{'metrics/precision(B)': 0.6091677395061049,
 'metrics/recall(B)': 0.6303558935279365,
 'metrics/mAP50(B)': 0.6193636477811901,
 'metrics/mAP50-95(B)': 0.3845747827008005,
 'fitness': 0.40805366920883945}

In [9]:
experiment_name = 'val'
work_name = 'raytune' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{work_name}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

best_model = YOLO("/home/under1/Detect/jeongui/sagittal/runs/detect/train/train_raytune/weights/best.pt")
best_model.val(name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'))

Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/under1/Detect/dataset/sagittal/labels/val.cache... 52 images, 0 backgrounds, 0 corrupt: 100%|██████████| 52/52 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  3.90it/s]


                   all         52         54      0.488      0.804      0.559       0.38
              negative         30         30      0.628      0.899      0.739       0.51
              positive         22         24      0.349      0.708      0.379       0.25
Speed: 0.2ms preprocess, 9.9ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/val/val_raytune2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f53bc11d3c0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [10]:
# test

experiment_name = 'test'
work_name = 'raytune' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{work_name}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

best_model.val(name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'), split="test")

Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)


val: Scanning /home/under1/Detect/dataset/sagittal/labels/test... 70 images, 0 backgrounds, 0 corrupt: 100%|██████████| 70/70 [00:00<00:00, 777.32it/s]

val: New cache created: /home/under1/Detect/dataset/sagittal/labels/test.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:01<00:00,  4.72it/s]


                   all         70         77      0.468      0.744      0.494      0.309
              negative         35         39      0.482      0.718      0.453      0.312
              positive         35         38      0.455       0.77      0.536      0.307
Speed: 0.2ms preprocess, 5.0ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs/detect/test/test_raytune2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f53d40a36a0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804